# 开始使用Keras函数式API

Keras函数式API是定义复杂模型（如多输出模型，有向无环图，或具有共享层的模型）的方法

## 全链接网络

Sequential模型可能是实现这种网络的一个更好选择，但这个例子能帮助我们进行一些简单的理解。
* 网络层的实例是可调用的，它以张量为参数，并且返回一个张量
* 输入输出均为张量，它们都可以用来定义一个模型(Model)
* 这样的模型同Keras的Sequential模型一样，都可以被训练

In [4]:
from keras.layers import Input, Dense
from keras.models import Model

In [5]:
#　这部分返回一个张量
inputs = Input(shape=(784,))

In [6]:
# 层的实例是可调用的，它以张量为参数，并且返回一个张量
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

In [8]:
#　这部分创建了一个包含输入层和三个全连接层的模型
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
# model.fit(data, label) #开始训练

## 所有的模型都可调用，就像网络层一样

利用函数式API，可以轻易地重用训练好的模型：可以将任何模型看做是一个层，然后通过传递一个张量来调用它。注意，在调用模型时，不仅重用模型的结构，还重用了它的权重。

In [9]:
x = Input(shape=(784,))
# 这是可行的，并且返回上面定义的10-way softmax
y = model(x)

这种方式能允许我们快速创建可以处理序列输入的模型。只需要一行代码，你就将图像分类模型转换为视频分类模型。

In [10]:
from keras.layers import TimeDistributed

In [11]:
# 输入张量是２０个时间步的序列
# 每一个时间为一个７８４维的向量

input_sequences = Input(shape=(20, 784))

# 这部分将我们之前定义的模型应用于输入序列中的每个时间步
# 之前定义的模型的输出是一个１０-way　softmax,
# 因而下面的层的输出将是维度为１０的２０个向量的序列

processed_sequences = TimeDistributed(model)(input_sequences)

## 多输入多输出模型
以下是函数式API的一个很好的例子：具有多个输入和输出的模型。函数式API使处理大量交织的数据流变得容易。
来考虑下面的模型。我们试图预测Twitter上的一条新闻标题有多少转发和点赞数。模型的主要输入将是新闻标题本省，即一系列词语，但是为了增添趣味。我们的模型还添加了其他的辅助输入来接收额外的数据，例如新闻标题的发布时间等。该模型也将通过两个损失函数进行监督学习。较早地在模型中使用主损失函数，是深度学习模型的一个良好正则方法。

主要输入接收新闻标题本身，即一个整数序列（每个整数编码一个词）。 这些整数在 1 到 10,000 之间（10,000 个词的词汇表），且序列长度为 100 个词。

In [12]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

In [13]:
# 标题输入：接收一个含有１００个整数的序列，每个整数在１到１００００之间
# 注意，通过传递一个"name" 参数来命名任何层
main_input = Input(shape=(100,), dtype='int32', name='main_input')

# Embedding 层将输入序列编码为一个稠密向量的序列
# 每个向量维度为５１２
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

# ＬＳＴＭ层把向量序列转换成单个向量
# 它包含整个序列的上下文信息
lstm_out = LSTM(32)(x)

在这里，我们插入辅助损失，使得即使在模型主损失很高的情况下，LSTM层和Embedding层能被平稳地训练

In [14]:
auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

In [20]:
import keras
auxiliary_input = Input(shape=(5,), name='aux_input')
x = keras.layers.concatenate([lstm_out, auxiliary_input])

In [21]:
# 堆叠多个全连接网络层
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# 最后添加主要的逻辑回归
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

In [22]:
model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])

现在编译模型，并给辅助损失分配一个０．２的权重。如果要为不同的输出指定不同的loss_weight或loss，可以使用列表或字典。在这里，我们给loss参数传递单个损失函数，这个损失将用于所有输出。

In [24]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
             loss_weight=[1., 0.2])

In [27]:
# model.fit([headline_data, additional_data], [labels, labels],
#          epochs=50, batch_size=32)

In [29]:
# model.compile(optimizer='rmsprop',
#               loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
#               loss_weights={'main_output': 1., 'aux_output': 0.2})

# # 然后使用以下方式训练：
# model.fit({'main_input': headline_data, 'aux_input': additional_data},
#           {'main_output': labels, 'aux_output': labels},
#           epochs=50, batch_size=32)